In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [2]:
# Set up the Selenium WebDriver to connect to the Selenium server (replace 'selenium' with your service name if different)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode to avoid opening a UI
driver = webdriver.Remote(
    command_executor='http://selenium:4444/wd/hub',
    options=options
)

In [3]:
# Go to the Letterboxd popular movies page
driver.get("https://letterboxd.com/films/popular/page/1/")

In [5]:
# Find all elements that contain movie links
movie_elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')

# Extract URLs from the elements
movie_urls = [element.get_attribute('href') for element in movie_elements]

In [6]:
movie_urls

['https://letterboxd.com/film/barbie/',
 'https://letterboxd.com/film/parasite-2019/',
 'https://letterboxd.com/film/everything-everywhere-all-at-once/',
 'https://letterboxd.com/film/fight-club/',
 'https://letterboxd.com/film/la-la-land/',
 'https://letterboxd.com/film/interstellar/',
 'https://letterboxd.com/film/oppenheimer-2023/',
 'https://letterboxd.com/film/joker-2019/',
 'https://letterboxd.com/film/dune-2021/',
 'https://letterboxd.com/film/pulp-fiction/',
 'https://letterboxd.com/film/whiplash-2014/',
 'https://letterboxd.com/film/spider-man-into-the-spider-verse/',
 'https://letterboxd.com/film/get-out-2017/',
 'https://letterboxd.com/film/the-batman/',
 'https://letterboxd.com/film/the-dark-knight/',
 'https://letterboxd.com/film/saltburn/',
 'https://letterboxd.com/film/the-truman-show/',
 'https://letterboxd.com/film/spider-man-across-the-spider-verse/',
 'https://letterboxd.com/film/lady-bird/',
 'https://letterboxd.com/film/american-psycho/',
 'https://letterboxd.com/f

In [1]:
import argparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

def scrape_movie_urls(num_movies):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    movie_urls = []
    page = 1
    while len(movie_urls) < num_movies:
        driver.get(f"https://letterboxd.com/films/popular/page/{page}/")
        elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')
        for element in elements:
            if len(movie_urls) >= num_movies:
                break
            movie_urls.append(element.get_attribute('href'))
        page += 1
    
    df = pd.DataFrame(movie_urls, columns=['url'])
    return df
    #df.to_csv('/app/data/movie_urls.csv', index=False)


In [2]:
x = scrape_movie_urls(5)
x

,url
0,https://letterboxd.com/film/barbie/
1,https://letterboxd.com/film/parasite-2019/
2,https://letterboxd.com/film/everything-everywh...
3,https://letterboxd.com/film/fight-club/
4,https://letterboxd.com/film/la-la-land/


In [3]:
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)

def scrape_reviews_per_movie(num_reviews_per_movie):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    movie_urls = pd.read_csv('/app/data/movie_urls.csv')['url'].tolist()
    all_reviews = []

    for movie_url in movie_urls:
        print(f"Scraping reviews for movie: {movie_url}")
        movie_reviews = []
        driver.get(f"{movie_url}/reviews/")
        
        page = 1
        while len(movie_reviews) < num_reviews_per_movie:
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
            
            for element in review_elements:
                if len(movie_reviews) >= num_reviews_per_movie:
                    break
                
                review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                review_text = review_text_element.text if review_text_element else None
                
                rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                star_rating = rating_element.text if rating_element else None
                
                if review_text and star_rating is not None:
                    movie_reviews.append({
                        'movie_url': movie_url,
                        'review': review_text,
                        'rating': normalize_rating(star_rating)
                    })
            
            # Go to next page if needed
            if len(movie_reviews) < num_reviews_per_movie:
                page += 1
                driver.get(f"{movie_url}/reviews/page/{page}/")
            else:
                break
        
        all_reviews.extend(movie_reviews)
    
    df = pd.DataFrame(all_reviews)
    return df

    #df.to_csv('/app/data/movie_reviews.csv', index=False)

In [12]:
y = scrape_movie_urls(1)

In [13]:
y

,url
0,https://letterboxd.com/film/barbie/


# Organize code a bit more

In [1]:
import argparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

## 1a) Scrape movie urls

In [3]:
def scrape_movie_urls(num_movies):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    movie_urls = []
    page = 1
    while len(movie_urls) < num_movies:
        driver.get(f"https://letterboxd.com/films/popular/page/{page}/")
        elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')
        for element in elements:
            if len(movie_urls) >= num_movies:
                break
            movie_urls.append(element.get_attribute('href'))
        page += 1
    
    df = pd.DataFrame(movie_urls, columns=['url'])
    return df
    #df.to_csv('/app/data/movie_urls.csv', index=False)

In [4]:
movie_urls = scrape_movie_urls(5)
movie_urls

,url
0,https://letterboxd.com/film/barbie/
1,https://letterboxd.com/film/parasite-2019/
2,https://letterboxd.com/film/everything-everywh...
3,https://letterboxd.com/film/fight-club/
4,https://letterboxd.com/film/la-la-land/


In [8]:
for movie_url in movie_urls.url:
    print(movie_url)

https://letterboxd.com/film/barbie/
https://letterboxd.com/film/parasite-2019/
https://letterboxd.com/film/everything-everywhere-all-at-once/
https://letterboxd.com/film/fight-club/
https://letterboxd.com/film/la-la-land/


## 1b) Scrape reviews from movie urls

In [11]:
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)

def scrape_reviews_from_movies(movie_urls, num_reviews_per_movie):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    all_reviews = []

    for movie_url in movie_urls.url:
        print(f"Scraping reviews for movie: {movie_url}")
        movie_reviews = []
        driver.get(f"{movie_url}/reviews/")
        
        page = 1
        while len(movie_reviews) < num_reviews_per_movie:
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
            
            for element in review_elements:
                if len(movie_reviews) >= num_reviews_per_movie:
                    break
                
                review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                review_text = review_text_element.text if review_text_element else None
                
                rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                star_rating = rating_element.text if rating_element else None
                
                if review_text and star_rating is not None:
                    movie_reviews.append({
                        'movie_url': movie_url,
                        'review': review_text,
                        'rating': normalize_rating(star_rating)
                    })
            
            # Go to next page if needed
            if len(movie_reviews) < num_reviews_per_movie:
                page += 1
                driver.get(f"{movie_url}/reviews/page/{page}/")
            else:
                break
        
        all_reviews.extend(movie_reviews)
    
    df = pd.DataFrame(all_reviews)
    return df

    #df.to_csv('/app/data/movie_reviews.csv', index=False)

In [12]:
movie_reviews = scrape_reviews_from_movies(movie_urls, num_reviews_per_movie=1)
movie_reviews

Attempting to connect to Selenium WebDriver...
Successfully connected to WebDriver!
Scraping reviews for movie: https://letterboxd.com/film/barbie/
Scraping reviews for movie: https://letterboxd.com/film/parasite-2019/
Scraping reviews for movie: https://letterboxd.com/film/everything-everywhere-all-at-once/
Scraping reviews for movie: https://letterboxd.com/film/fight-club/
Scraping reviews for movie: https://letterboxd.com/film/la-la-land/


,movie_url,review,rating
0,https://letterboxd.com/film/barbie/,Literally made me cry in theaters. Margot was ...,0.8
1,https://letterboxd.com/film/parasite-2019/,"o filme é bom, mas no final deixa a desejar… a...",0.6
2,https://letterboxd.com/film/everything-everywh...,a film that just can’t be described…???,0.7
3,https://letterboxd.com/film/fight-club/,You know when you just have to sit and reevalu...,0.8
4,https://letterboxd.com/film/la-la-land/,estoy destruida,1.0


## 2A) Scrape from users

In [1]:
import argparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

def scrape_user_urls(num_users):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("...Connected to selenium service!")
    user_urls = []
    page = 1
    while len(user_urls) < num_users:
        driver.get(f"https://letterboxd.com/members/popular/this/week/page/{page}/")
        elements = driver.find_elements(By.CSS_SELECTOR, 'a.name')
        for element in elements:
            if len(user_urls) >= num_users:
                break
            user_urls.append(element.get_attribute('href'))
        page += 1
    
    df = pd.DataFrame(user_urls, columns=['url'])
    print("User urls sucessfully scraped!\nUser urls preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium webdriver...")
    driver.quit()
    print("Connection closed sucessfully!\n\n")
    return df


In [2]:
user_urls = scrape_user_urls(10)

Attempting to connect to Selenium WebDriver...
...Connected to selenium service!
User urls sucessfully scraped!
User urls preview:
                                    url
0      https://letterboxd.com/kurstboy/
1  https://letterboxd.com/schaffrillas/
2           https://letterboxd.com/jay/
3     https://letterboxd.com/jaragon23/
4    https://letterboxd.com/deathproof/

Closing connection to selenium webdriver...
Connection closed sucessfully!




## 2b) Scrape reviews from user urls

In [31]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
print("Attempting to connect to Selenium WebDriver...")
driver = webdriver.Remote(
    command_executor='http://selenium:4444/wd/hub',
    options=options
)
print("Successfully connected to WebDriver!")

Attempting to connect to Selenium WebDriver...
Successfully connected to WebDriver!


In [32]:
user_url = user_urls.url[0] ; page=1
driver.get(f"{user_url}/films/reviews/by/added/page/{page}/")
element = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')[0]
review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
review_text = review_text_element.text if review_text_element else None

rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
star_rating = rating_element.text if rating_element else None

In [35]:
element.find_element(By.CSS_SELECTOR, "h2.headline-2 a").text

'Labyrinth'

In [17]:
driver.close()

In [ ]:
<a href="/film/alien-resurrection/" class="frame has-menu"><span class="frame-title">Alien Resurrection (1997)</span><span class="overlay"></span><span class="overlay-actions js-film-options -w70" style="display: none;"><span class="film-watch-link-target" data-film-id="47887"><span class="film-watch-link"><span class="has-icon icon-16 icon-watch ajax-click-action" data-action="/film/alien-resurrection/mark-as-watched/"><span class="replace icon"></span>Seen this film?</span></span></span><span class="like-link-target" data-likeable-uid="film:47887"><span class="like-link"><span class="has-icon icon-16 ajax-click-action  icon-like" data-action="/s/film:47887/like/" data-recaptcha-action="film_like" data-confirm="" data-modal-confirm="true"><span class="icon"></span>Like this film?</span></span></span><span class="replace menu-link icon"></span></span></a>

In [3]:
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)

def scrape_reviews_from_users(user_urls, num_reviews_per_user):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    all_reviews = []

    for user_url in user_urls.url:
        print(f"Scraping reviews for user: {user_url}")
        user_reviews = []
        page = 1
        while len(user_reviews) < num_reviews_per_user:
            driver.get(f"{user_url}/films/reviews/by/added/page/{page}/")
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
            
            for element in review_elements:
                if len(user_reviews) >= num_reviews_per_user:
                    break
                
                review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                review_text = review_text_element.text if review_text_element else None
                
                rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                star_rating = rating_element.text if rating_element else None
                
                if review_text and star_rating is not None:
                    user_reviews.append({
                        'user_url': user_url,
                        'review': review_text,
                        'rating': normalize_rating(star_rating)
                    })
            
            # Go to next page if needed
            if len(user_reviews) < num_reviews_per_user:
                page += 1
            else:
                break
        
        all_reviews.extend(user_reviews)
    
    df = pd.DataFrame(all_reviews)
    print("Reviews sucessfully scraped from user urls!\nReviews preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium webdriver...")
    driver.quit()
    print("Connection closed sucessfully!\n\n")
    return df

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_users', type=int, default=10, help='Number of user URLs to scrape')
    parser.add_argument('--num_reviews_per_user', type=int, default=10, help='Number of reviews to scrape per user')
    args = parser.parse_args()
    
    user_urls = scrape_user_urls(args.num_users)
    reviews_from_users = scrape_reviews_from_users(user_urls, args.num_reviews_per_user)
    reviews_from_users.to_csv('/app/data/reviews_from_users.csv', index=False)